In [26]:
import sportsdataverse as sdv
import pandas as pd
import polars

In [17]:
mbb_df = sportsdataverse.mbb.load_mbb_team_boxscore(seasons=range(2023,2024))

100%|██████████| 1/1 [00:00<00:00,  2.35it/s]


In [40]:
mbb_df.head()

,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,...,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score
0,401522202,2023,3,2023-04-03,2023-04-03 21:20:00-04:00,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,...,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,76
1,401522202,2023,3,2023-04-03,2023-04-03 21:20:00-04:00,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,...,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,59
2,401522201,2023,3,2023-04-01,2023-04-01 21:16:00-04:00,2390,s:40~l:41~t:2390,miami-hurricanes,Miami,Hurricanes,...,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,72
3,401522201,2023,3,2023-04-01,2023-04-01 21:16:00-04:00,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,...,miami-hurricanes,Miami,Hurricanes,MIA,Miami Hurricanes,Miami,004325,f0f0f0,https://a.espncdn.com/i/teamlogos/ncaa/500/239...,59
4,401522200,2023,3,2023-04-01,2023-04-01 18:09:00-04:00,2226,s:40~l:41~t:2226,florida-atlantic-owls,Florida Atlantic,Owls,...,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,72


In [31]:
mbb_df = mbb_df.to_pandas()

In [41]:
mbb_df.head()

,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,...,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score
0,401522202,2023,3,2023-04-03,2023-04-03 21:20:00-04:00,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,...,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,76
1,401522202,2023,3,2023-04-03,2023-04-03 21:20:00-04:00,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,...,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,59
2,401522201,2023,3,2023-04-01,2023-04-01 21:16:00-04:00,2390,s:40~l:41~t:2390,miami-hurricanes,Miami,Hurricanes,...,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,72
3,401522201,2023,3,2023-04-01,2023-04-01 21:16:00-04:00,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,...,miami-hurricanes,Miami,Hurricanes,MIA,Miami Hurricanes,Miami,004325,f0f0f0,https://a.espncdn.com/i/teamlogos/ncaa/500/239...,59
4,401522200,2023,3,2023-04-01,2023-04-01 18:09:00-04:00,2226,s:40~l:41~t:2226,florida-atlantic-owls,Florida Atlantic,Owls,...,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,72


In [37]:
mbb_df.columns

Index(['game_id', 'season', 'season_type', 'game_date', 'game_date_time',
       'team_id', 'team_uid', 'team_slug', 'team_location', 'team_name',
       'team_abbreviation', 'team_display_name', 'team_short_display_name',
       'team_color', 'team_alternate_color', 'team_logo', 'team_home_away',
       'team_score', 'team_winner', 'assists', 'blocks', 'defensive_rebounds',
       'field_goal_pct', 'field_goals_made', 'field_goals_attempted',
       'flagrant_fouls', 'fouls', 'free_throw_pct', 'free_throws_made',
       'free_throws_attempted', 'largest_lead', 'offensive_rebounds', 'steals',
       'team_turnovers', 'technical_fouls', 'three_point_field_goal_pct',
       'three_point_field_goals_made', 'three_point_field_goals_attempted',
       'total_rebounds', 'total_technical_fouls', 'total_turnovers',
       'turnovers', 'opponent_team_id', 'opponent_team_uid',
       'opponent_team_slug', 'opponent_team_location', 'opponent_team_name',
       'opponent_team_abbreviation', 'oppon

In [43]:
date = 

mbb_df[]

0         7
1        13
2        10
3        19
4         6
         ..
12435     8
12436    13
12437    27
12438     8
12439    13
Name: assists, Length: 12440, dtype: int32

In [45]:
type(mbb_df['game_date']

0       2023-04-03
1       2023-04-03
2       2023-04-01
3       2023-04-01
4       2023-04-01
           ...    
12435   2022-11-07
12436   2022-11-07
12437   2022-11-07
12438   2022-11-07
12439   2022-11-07
Name: game_date, Length: 12440, dtype: datetime64[ms]